# Homework 2

<b>ผู้ทำ</b>
- นายจรัญวัฒน์ บัวศรี 63340500005
- นายธนกฤต เมืองสุวรรณ 63340500020

<b>จุดประสงค์</b><br>
ในการบ้านนี้ ผู้จัดทำจะต้องทำการวิเคราะห์หาผู้ที่จะยังคงใช้บัตรและผู้ที่จะยกเลิกการใช้บัตรเครดิต จากข้อมูลของผู้ใช้บัตรว่าบุคคลไหนจะมีแนวโน้มยกเลิกบัตรเครดิต โดยใช้ข้อมูลจาก credit_card_churn ในการทำนาย โดยสิ่งที่ต้องทำคือทำนายว่าลูกค้าแบบไหนที่จะยกเลิกการใช้บัตรเครดิตเพื่อให้ความสนใจกับลูกค้าประเภทนั้น และให้ความสำคัญในกาหาเหตุผลที่ลูกค้าจะยกเลิกบัตรเครดิต

## Data exploration

### Load dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv("credit_card_churn.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'credit_card_churn.csv'

ตรวจสอบ dataset ว่ามีข้มูลอะไรบ้าง

### Input Variables :
1. CLIENTNUM
2. Customer_Age
3. Gender
4. Dependent_count
5. Education_Level
6. Marital_Status
7. Income_Category
8. Card_Category
9. Months_on_book
10. Total_Realationship_Count
11. Months_Inactive_12_mon
12. Contacts_Count_12_mon
13. Credit_Limit
14. Total_Revolving_Bal
15. Avg_Open_To_Buy
16. Total_Amt_Chng_Q4_Q1
17. Total_Trans_Amt
18. Total_Trans_Ct
19. Total_Ct_Chng_Q4_Q1
20. Avg_Utilization_Ratio
21. Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1
22. Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2
### Output variable :
22. Attrition_Flag 

In [ ]:
df.columns

In [ ]:
sns.pairplot(df,hue="Attrition_Flag")

Drop columns ที่เป็น ชื่อ และ data leakage

In [ ]:
df.drop(columns=["CLIENTNUM", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1", "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"], inplace=True)

In [ ]:
df

ตรวจสอบชนิดของข้อมูลแต่ละ column และค่า null

In [ ]:
df.info()

### Visualization

ใช้ pairplot เพื่อดู pattern ของ class เมื่อนำ feature แต่ละตัวมาเะปรียบเทียบกัน

In [ ]:
sns.pairplot(df,hue="Attrition_Flag")

In [ ]:
df.describe()

### แยกชนิดของข้อมูลเป็น continues value, discrete value และ categorical value

In [ ]:
numerical_columns = list(df.select_dtypes('float64').columns) + list(df.select_dtypes('int64').columns)
categorical_columns = list(df.select_dtypes('object').columns)

In [ ]:
numerical_columns

In [ ]:
categorical_columns

In [ ]:
discrete_columns = ["Dependent_count", "Total_Relationship_Count", "Months_Inactive_12_mon", "Contacts_Count_12_mon"]
contineous_columns = [e for e in numerical_columns if e not in discrete_columns]

In [ ]:
discrete_columns

In [ ]:
contineous_columns

In [ ]:
compare_df = df[contineous_columns]
compare_df = pd.concat((compare_df,df["Attrition_Flag"]),axis=1)

In [ ]:
compare_df

### Visualiztion เพื่อดูขอบเขตและ density ของแต่ละ class

In [ ]:
sns.pairplot(compare_df,hue="Attrition_Flag")

In [ ]:
sns.pairplot(compare_df,hue="Attrition_Flag", kind='kde')

จากการใช้ pairplot จะพบว่า feature บางตัวเมื่อนำมาเปรียบเทียบกันก็จะสามารถจัดกลุ่มข้อมูลเดียวกันให้มากระจุกตัวเป็นกลุ่มเดียวกันและจะเห็น density ของข้อมูลแต่ละ class ว่าอยู่ห่างกันยังไงบ้าง โดยเริ่มจากการเปรียบเทียบ continuous value เทียบกับ continuous value เช่น
- custom_age เทียบกับ total_amt_change_Q4_Q1, total_trans_amt, total_trans_ct และ total_change_ct
- mounts_on_book เทียบกับ total_amt_change_Q4_Q1, total_trans_amt, total_trans_ct และ total_change_ct
- credit_limit เทียบกับ total_amt_change_Q4_Q1, total_trans_amt, total_trans_ct และ total_change_ct
- total_revenue_bal เทียบกับ total_amt_change_Q4_Q1, total_trans_amt, total_trans_ct และ total_change_ct
- avg_open_to_buy เทียบกับ total_amt_change_Q4_Q1, total_trans_amt, total_trans_ct และ total_change_ct
- total_amt_change_Q4_Q1 เทียบกับ total_ct_change_Q4_Q1, total_change_ct และ total_amt_change_Q4_Q1
- total_trans_amt เทียบกับ total_ct_change_Q4_Q1, total_trans_ct, total_amt_change_Q4_Q1, avg_open_to_buy, total_revenue_bal และ credit_limit

ซึ่งจะเห็นว่า feature ที่สามารถแบ่งกลุ่มข้อมูลที่ค่อนข้างชัดเจนจะได้แก่ total_amt_change_Q4_Q1, total_amt_change_Q4_Q1, total_trans_amt, total_trans_ct, total_ct_change_Q4_Q1 และ avg_open_to_buy
ถึงแม้ว่าข้อมูลมากระจุกตัวเป็นกลุ่มก้อน แต่ข้อมูลไม่สามารถถูกแบ่งได้ด้วยการใช้เส้นแบ่งธรรมดาเนื่องจากข้อมูลส่วนใหญ่มีการซ้อนทับกันซึ่งอาจจะค้องใช้ classifier ที่สามารถเปลี่ยน hyperplane ของการแสดงผลข้อมูลในการวิเคราะห์<br>
เราจึงตั้งสมมติฐานกับข้อมูลที่ได้ดังนี้
- คนที่ยกเลิกบัตรเครดิตมักจะมีข้อมูล total_amt_change_Q4_Q1 น้อยกว่า 1
- คนที่ยกเลิกบัตรเครดิตมักจะมีข้อมูล total_ct_change_Q4_Q1 น้อยกว่า 2
- คนที่ยกเลิกบัตรเครดิตมักจะมีข้อมูล total_trans_amt น้อยกว่า 15000
- คนที่ยกเลิกบัตรเครดิตมักจะมีข้อมูล total_trans_ct น้อยกว่า 100
- ต้องใช้ model หรือแปลงข้อมูลให้อยู่ในปริภูมิที่ที่ทำให้สามารถแยก classs ได้

### เปรียบเทียบข้อมูล categorical และ continuous

#### ข้อมูลอายุเทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Customer_Age", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Customer_Age", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Customer_Age", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Customer_Age", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Customer_Age", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Customer_Age", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Customer_Age", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Customer_Age", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Customer_Age", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Customer_Age", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Customer_Age กับ categorical ได้ผลดังนี้
- จากกราฟพบว่าการกระจายตัวของอายุทั้งเพศชายและเพศหญิงมีลักษณะเหมือนกัน ซึ่งแสดงให้เห็นว่าไม่ว่าเพศไหนอายุเท่าไรไม่สามารถบอกได้ว่าจะถือหรือยกเลิกบัตร
- จากกราฟพบว่าการกระจายตัวของการศึกษามีลักษณะเหมือนกัน ทำให้ไม่ส่าผู้นั้นจะจบการศึกษามาระดับไหน ก็ไม่สามารถบอกได้ว่าเขาจะถือหรือยกเลิกบัตรเครดิต
- จากกราฟ boxplot พบว่าการกระจายตัวของอายุในแต่ละการศึกษามีลักษณะเหมือนกัน และจะเห็นได้ว่าอายุของคนที่จบการศึกษาในแต่ระดับไม่สามารถแยก class ได้อย่างชัดเจน
- จากกราฟ boxplot พบว่าการกระจายตัวของอายุในแต่ละช่วงเงินเดือนมีลักษณะเหมือนกัน และบว่าการเปรียบเทียบอายุกับเงินเดือนไม่สามารถบ่งบอกได้ถึงแนวโน้มการยกเลิกบัตรเครดิต
- จากการฟพบว่าถึงแม้บัตรประเภท platinum จะมีจำนวนผู้ที่ถือน้อยที่สุดแต่เมื่อนำมาพิจารณาร่วมกับอายุพบว่าผผู้ที่มีอายุมากที่ถือบัตร Platinum มักมีแนวโน้มจะยกเลิกบัตร

#### ข้อมูล Credit limit เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Credit_Limit", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Credit_Limit", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Credit_Limit", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Credit_Limit", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Credit_Limit", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Credit_Limit", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Credit_Limit", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Credit_Limit", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Credit_Limit", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Credit_Limit", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ credit limit กับ categorical ได้ผลดังนี้
- จากการฟจะพบว่าข้อมูล credit limit ของแต่ละเพศมีการเบ้ขวา ซึ่งกราฟแสดงให้เห็นว่าเพศชายมี credit limit ที่สูงกว่าเพศหญิงได้ชัดแต่ข้อมูลกราฟการกระจาตัวของผู้ที่ถือบัตรเครดิตและยกเลิกบัตรเครดิตมีความเหมือนกันในแต่ละเพศทำให้ไม่สามารถประเภทได้อย่างชัดเจน
- จากการฟจะพบว่าข้อมูล credit limit ของแต่ละระดับการศึกษาถ้ามีค่ามากก็จะมีแนวโน้มที่คนจะถือบัตรเครดิต แต่กราฟการกระจายตัวมีความใกล้เคียงกันทำให้ไม่สามารถแยก class  ได้อย่างชัดเจน
- จากการฟจะพบว่าข้อมูล credit limit ของแต่ละสถานะการแต่งงาน พบว่าการกระจายตัวมีความเหมือนกัน จึงสรุปได้ว่าแต่ละสถานะทั้งผู้ที่ถือบัตรและไม่ถือบัตรก็จะมี credits limit ที่ไม่แตกต่างกัน
- จากการฟพบว่าคนที่มีเงินเดือนน้อยกว่า 40K มีจำนวนมากที่สุดแต่จากการฟจะเห็นว่าคนกลุ่มนี้มีค่า credit limit ที่ใกล้เคียงกัน โดยแนวโน้มคนที่ใช้บัตรมันจะมีค่าเฉลี่ย credit limit สูงกว่าคนที่ยกเลิกบัตรในแต่ละ class
- จากการฟจะพบว่าข้อมูลประเภทของบัตรที่ถูกถือมากที่สุดคือประเภท Blue จากนั้นจึงนำไปหา distribution ของบัตรแต่ละประเภทพบว่า บัตรประเภท Blue Gold Silver มีลักษณะการกระจายตัวใกล้เคียงกันของทั้ง 2 class แต่สำหรับการ์ดประเภท Platinum จะเห็นได้ว่าผู้ที่ถือบัตรจะมีแนวโน้มที่ credits limit สูงกว่าของผู้ที่ยกเลิกบัตรอย่างชัดเจนถึงแม้ว่าจะมีจำนวนผู้ถือบัตรน้อยที่สุด

#### ข้อมูล Avg_Open_To_Buy เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Open_To_Buy", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Open_To_Buy", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Open_To_Buy", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Open_To_Buy", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Open_To_Buy", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Open_To_Buy", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Open_To_Buy", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Open_To_Buy", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Open_To_Buy", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Open_To_Buy", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Avg_Open_To_Buy กับ categorical ได้ผลดังนี้
- จากากราฟพบว่าบุคคลที่มีเงินเดือนต่ำกว่า 80K จะมีค่า Avg_Open_To_buy ที่ต่ำกว่าบุคลที่มีเงินเดือนแบบอื่นๆและมีแนวโน้มที่จะยกเลิกบัตรเครดิตมากว่าฐานเงินเดือนอื่นๆเมื่อพิจารณาจากค่าเฉลี่ยและที่บุคคลที่ทีเงินเดือนมากกว่า 80K จะมีแนวโน้มที่คนจะใช้บัตรเครดิตมากว่ายกเลิกแม้ว่าจะไม่สามารถแยกได้อย่างชัดเจน
- จากกราฟพบว่าสถานะการแต่งงานของผู้ที่ถือบัตรและยกเลิกบัตรมีการกระจายตัใกล้เคียงกันเมื่อพิจารณาจาก Avg_Open_To_Buy ซึ่งไม่สามารถแยก class ได้ชัดเจน ซึ่งสามารถตีความข้อมูลได้คือ Avg_Open_To_Buy ของสถานะแต่ละแบบไม่สามารถแยก class ได้
- จากกราฟพบว่าในแต่ละระดับการศึกษา ผู้ที่ถือบัตรและยกเลิกบัตรมีการกระจายตัวใกล้เคียงกันเมื่อพิจารณาจาก Avg_Open_To_Buy ทำให้บอกได้ว่า Avg_Open_To_buy ในแต่ระดับไม่สามารถบอกได้ถึงแนวโน้มของผู้ถือบัตรและยกเลิกบัตร
- จากกราฟพบว่าจำนวนผู้ถือบัตรและยกเลิกบัตรของทั้ง 2 เพศมีจำนวนใกล้เคียงกัน แต่เมื่อตรวจสอบจาก boxplot จะพบว่าผู้หญิงจะมี Avg_Open_To_Buy ต่ำกว่าผู้ชายแต่ไม่สามารถแยกความแตกต่างระหว่าง class เนื่องจาก Avg_Open_To_buy ของแต่ละเพศทั้งทั้งผู้ถือบัตรและยกเลิกบัตร
- จากกราฟพบว่าประเภทบัตรเทียบกับ Avg_Open_To_Buy บัตรอื่นๆในที่ไม่ใช่ Platimum ขะพบว่าจะแยกคนที่ใช้บัตรกับยกเลิกบัตรไม่ชัดเจน แต่ในประเภทบัตร platinum จะเห็นได้ว่าผู้ที่มี Avg_Open_To_Buy ต่ำจะมีแนวโน้มที่ยกเลิกบัตร

#### ข้อมูล Total_Amt_Chng_Q4_Q1 เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Amt_Chng_Q4_Q1", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Total_Amt_change_Q4_Q1 กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิงมีการกระจายตัวไม่แตกต่างกัน และโดยคนที่ใช้บัตรเครดิตจะมีค่าเฉลี่ย Total_Amt_change_Q4_Q1 สูงกว่าคนที่ใช้ยกเลิกเครดิต จึงสรุปได้ว่าคนที่มี Total_Amt_change_Q4_Q1 สูงจะมีแนวโน้มใช้บัตรเครกิตไม่ว่าจะเพศไหนก็ตามถึงแม้ว่าแนวโน้มจะไม่ชัดเจน
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าในแต่ละระดับการศึกษาคนที่มี Total_Amt_change_Q4_Q1 สูงจะมีแนวโน้มที่ใช้บัตรมากกว่าถึงแม้แนวโน้มจะไม่ชัดเจน
- เมื่อเปรียบเทียบกับสถานะการแต่งงาน พบว่าในแต่ละสถานะมีการกระจายตัวคล้ายกันและพบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Amt_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร และสรุปได้ว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Amt_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตรโดยสถานะการแต่งงานไม่ส่งผลต่อการใช้บัตร
- เมื่อเปรียบเทียบกับเงินเดือนพบว่าการกระจายตัวของแต่ละเงินเดือนมีลักษณะเหมือนกันและพบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Amt_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร และสรุปได้ว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Amt_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตรโดยเงินเดือนไม่ส่งผลต่อการใช้ขัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าการกระจายตัวของแต่ละประเภทบัตรมีลักษณะเหมือนกัน และพบว่าค่า Total_Amt_change_Q4_Q1 ไม่สามารถแยกคนที่ใช้กับยกเลิกบัตรได้อย่างชัดเจนเมื่อนำมาใช้คู่กับประเภทบัตร

#### ข้อมูล Total_Ct_Chng_Q4_Q1 เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Ct_Chng_Q4_Q1", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Total_Ct_Change_Q4_Q1 กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิงมีการกระจายตัวไม่แตกต่างกัน และโดยคนที่ใช้บัตรเครดิตจะมีค่าเฉลี่ย Total_Ct_Change_Q4_Q1 สูงกว่าคนที่ใช้ยกเลิกเครดิต จึงสรุปได้ว่าคนที่มี Total_Ct_Change_Q4_Q1 สูงจะมีแนวโน้มใช้บัตรเครกิตไม่ว่าจะเพศไหนก็ตาม
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าในแต่ละระดับการศึกษาคนที่มี Total_Ct_Change_Q4_Q1 สูงจะมีแนวโน้มที่ใช้บัตรมากกว่า
- เมื่อเปรียบเทียบกับสถานะการแต่งงาน พบว่าในแต่ละสถานะมีการกระจายตัวคล้ายกันและพบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Ct_Change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร และสรุปได้ว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Ct_Change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตรโดยสถานะการแต่งงานไม่ส่งผลต่อการใช้บัตร
- เมื่อเปรียบเทียบกับเงินเดือนพบว่าการกระจายตัวของแต่ละเงินเดือนมีลักษณะเหมือนกันและพบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Ct_Change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร และสรุปได้ว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Ct_Change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตรโดยเงินเดือนไม่ส่งผลต่อการใช้ขัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าการกระจายตัวของแต่ละประเภทบัตรมีลักษณะเหมือนกัน และพบว่าค่าเฉลี่ย Total_Ct_Change_Q4_Q1 ของคนใช้บัตรจะสูงกว่าคนยกเลิกบัตรไม่ว่าจะใช้บัตรประเ๓ทไหนก็ตาม

#### ข้อมูล Avg_Utilization_Ratio เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Utilization_Ratio", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Utilization_Ratio", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Utilization_Ratio", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Utilization_Ratio", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Utilization_Ratio", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Utilization_Ratio", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Utilization_Ratio", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Utilization_Ratio", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Avg_Utilization_Ratio", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Avg_Utilization_Ratio", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Avg_utilization_Ratio กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิง พบว่าเพศหญิงขะมีช่วงและค่าเฉลี่ยของ และโดยคนที่ใช้บัตรเครดิตจะมีค่าเฉลี่ย Avg_utilization_Ratio สูงกว่าเพศชายและคนใช้บัตรจะมีค่าเฉลี่ย Avg_utilization_Ratio สูงกว่าคนที่ยกเลิกบัตรในทุกเพศ จึงสรุปได้ว่าคนที่มี Avg_utilization_Ratio สูงจะมีแนวโน้มใช้บัตรเครกิตไม่ว่าจะเพศไหนก็ตาม
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าการกระจายตัวของแต่ละระดับจะมีลักษณะเดียวกันและในแต่ละระดับการศึกษาคนที่มี Avg_utilization_Ratio สูงจะมีแนวโน้มที่ใช้บัตรมากกว่า
- เมื่อเปรียบเทียบกับสถานะการแต่งงาน พบว่าในแต่ละสถานะมีการกระจายตัวคล้ายกันและพบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Avg_utilization_Ratio สูงกว่าคนที่ยกเลิกบัตร และสรุปได้ว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Avg_utilization_Ratio สูงกว่าคนที่ยกเลิกบัตรโดยสถานะการแต่งงานไม่ส่งผลต่อการใช้บัตร
- เมื่อเปรียบเทียบกับเงินเดือน พบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Avg_utilization_Ratio สูงกว่าคนที่ยกเลิกบัตร และสรุปได้ว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Avg_utilization_Ratio สูงกว่าคนที่ยกเลิกบัตรโดยเงินเดือนไม่ส่งผลต่อการใช้ขัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าประเภทบัตรที่ไม่ใช่ platinum จะไม่สามารถแยกคนใช้และคนยกเลิกบัตรเมื่อเทียบกับ Avg_utilization_Ratio และจากบัตรทุกประเภทค่าเฉลี่ย Avg_utilization_Ratio ของคนใช้บัตรจะสูงกว่าคนยกเลิกบัตร

#### ข้อมูล Months_on_book เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_on_book", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_on_book", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_on_book", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_on_book", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_on_book", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_on_book", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_on_book", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_on_book", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_on_book", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_on_book", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Months_on_book กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิง พบว่าเพศทั้งสองมีลักษณะการกระจายตัวเหมือนกันและไม่สามารถแยกคนที่ใช้บัตรและยกเลิกบัตรได้
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าการกระจายตัวของแต่ละระดับจะมีลักษณะคล้ายกันและในแต่ละระดับการศึกษาคนที่มีค่าเฉี่ย Months_on_book ของคนที่ใช้บัตรและยกเลิกบัตรมีค่าไม่ต่างกัน
- เมื่อเปรียบเทียบสถานะการแต่งงาน พบว่าการกระจายตัวของแต่ละสถานะจะมีลักษณะคล้ายกันและในแต่ละสถานะคนที่มีค่าเฉี่ย Months_on_book ของคนที่ใช้บัตรและยกเลิกบัตรมีค่าไม่ต่างกัน
- เมื่อเปรียบเทียบกับเงินเดือน พบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Months_on_book ไม่ต่างจากคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าบัตรแต่ละประเภทมีการกระจายตัวแบบ normal distribution โดยบัตร Gold คนที่ใช้บัตรจะมีค่าเฉลี่ย Months_on_book สูงกว่าคนที่ยกเลิกบัตร และบัตร Platinum คนที่ใช้บัตรจะมีค่าเฉลี่ย Months_on_book ต่ำกว่าคนที่ยกเลิกบัตร

#### ข้อมูล Total_Revolving_Bal เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Revolving_Bal", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Revolving_Bal", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Revolving_Bal", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Revolving_Bal", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Revolving_Bal", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Revolving_Bal", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Revolving_Bal", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Revolving_Bal", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Revolving_Bal", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Revolving_Bal", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Total_Revolving_Bal กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิง พบว่าเพศทั้งสองมีลักษณะการกระจายตัวเหมือนกันและคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Revolving_Bal สูงกว่าคนที่ยกเลิกบัตรอย่างชัดเจน
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าการกระจายตัวของแต่ละระดับจะมีลักษณะเหมือนกันและในแต่ละระดับการศึกษาคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Revolving_Bal สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบสถานะการแต่งงาน พบว่าการกระจายตัวของแต่ละสถานะจะมีลักษณะคล้ายกันและคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Revolving_Bal สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบเงินเดือน พบว่าการกระจายตัวของแต่เงินเดือนจะมีลักษณะคล้ายกันและคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Revolving_Bal สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าผู้ที่ใช้บัตรจะจะมีค่าเฉลี่ย Total_Revolving_Bal สูงกว่าคนที่ยกเลิกบัตร โดยจะสามารถแยกได้อย่างชัดเจนเมื่อเป็นบัตร platinum

#### ข้อมูล Total_Trans_Amt เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Amt", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Amt", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Amt", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Amt", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Amt", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Amt", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Amt", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Amt", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Amt", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Amt", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Total_Trans_Amt กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิง พบว่าเพศชายมีการกระจายตัวของคนที่ใช้และยกเลิกบัตรใกล้เคียงกันแต่ในเพศหญิงพบว่าผู้หญิงที่ใช้บัตรจะมีค่าเฉลี่ย Total_Trans_Amt สูงกว่าผู้หญิงที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าการกระจายตัวของแต่ละระดับจะมีลักษณะเหมือนกันและในแต่ละระดับการศึกษาคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Trans_Amt สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบสถานะการแต่งงาน พบว่าการกระจายตัวของแต่ละสถานะจะมีลักษณะคล้ายกันและคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Trans_Amt สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบเงินเดือน พบว่าการกระจายตัวของแต่เงินเดือนจะมีลักษณะคล้ายกันและคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Trans_Amt สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าในบัตร Blue Gold และ Platinum ผู้ที่ใช้บัตรจะจะมีค่าเฉลี่ย Total_Trans_Amt สูงกว่าคนที่ยกเลิกบัตร โดยจะสามารถแยกได้อย่างชัดเจนเมื่อเป็นบัตร platinum

#### ข้อมูล Total_Trans_Ct เทียบกับ categorical ต่างๆ

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Gender",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Ct", y="Gender", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Ct", y="Gender", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Education_Level",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Ct", y="Education_Level", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Ct", y="Education_Level", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Marital_Status",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Ct", y="Marital_Status", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Ct", y="Marital_Status", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Income_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Ct", y="Income_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Ct", y="Income_Category", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Card_Category",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Trans_Ct", y="Card_Category", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Trans_Ct", y="Card_Category", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Total_Trans_Ct กับ categorical ได้ผลดังนี้
- เมื่อเทียบกับเพศผู้ใช้งานทั้งชายและหญิง พบว่าเพศชายมีการกระจายตัวของ Total_Trans_Ct มากกว่าเพศหญิง และทั้งสองเพศคนที่ใช้บัตรจะมีค่าเฉลี่ย Total_Trans_Ct สูงกว่าคนที่ไม่ใช้บัตร
- เมื่อเปรียบเทียบกับระดับการศึกษา พบว่าการกระจายตัวของแต่ละระดับจะมีลักษณะเหมือนกันยกเว้น Married และในแต่ละระดับการศึกษาคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Trans_Ct สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบสถานะการแต่งงาน พบว่าการกระจายตัวของแต่ละสถานะจะมีลักษณะคล้ายกันและคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Trans_Ct สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบเงินเดือน พบว่าการกระจายตัวของแต่เงินเดือนจะมีลักษณะคล้ายกันและคนที่ใช้บัตรมีแนวโน้มค่าเฉลี่ย Total_Trans_Ct สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับประเภทบัตรพบว่าคนที่ใช้บัตรจะมีค่า Total_Trans_Ct สูงกว่าคนที่ยกเลิกบัตร โดยเฉพาะบัตรประเภท Platinum 

### เปรียบเทียบข้อมูล categorical และ discrete

#### เปรียบเทียบ Dependent_count

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Customer_Age", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Customer_Age", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Credit_Limit", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Credit_Limit", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Months_on_book", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Months_on_book", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Dependent_count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Dependent_count", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Dependent_count", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Dependent_count กับ numerical ได้ผลดังนี้
- เมื่อเทียบกับอายุพบว่าเมื่อนำ Dependent_count ค่าต่างๆจะมีการกระจายตัวของอายุเป็น normal distribution ทั้งของคนที่ใช้บัตรและยกเลิกบัตรเครดิต
- เมื่อเปรียบเทียบกับcredits_limit พบว่าการกระจายตัวของแต่ละ Dependent_count จะมีลักษณะเหมือนและในขณที่ค่า Dependent_count 1-5 แสดงให้เห็นว่าผู่ที่ยกเลิกบัตรเครดิตมีแนวโน้มค่าเฉลี่ยของของ credit_limits ที่มากกว่าคนที่ใช้บัตร ยกเว้นค่า Dependent_count 0 ซึ่งอาจจะทำให้ค่าบางค่าใน freature ไม่ได้ใช้ weight เดียวกันกับค่าอื่น
- เมื่อเปรียบเทียบกับ Avg_Open_To_Buy พบว่ามีกระจายตัวใกล้เคียงกันและค่าเฉลี่ยของ Avg_Open_To_Buy มีค่าใกล้เคียงกันทั้งคนที่ใช้บัตรและยกเลิกบัตรทำให้ไม่สามารถแยกได้อย่างชัดเจน
- เมื่อเปรียบเทียบ Total_Amt_change_Q4_Q1 พบว่ามีการกระจายตัวใกล้เคียงกันในทุกๆค่าของ Dependent_count และกระจายตัวแบบ normal distribution
- เมื่อเปรียบเทียบ Total_ct_change_Q4_Q1 พบว่ามีการกระจายตัวของคนที่ใช้บัตรและยกเลิกบัตรเป็น normal distribution และเมื่อพิจารณาจากฐานนิยมและค่าเฉลี่ยจะสามารถแยกคนที่ใช้บัตรและยกเลิกบัตรได้ค่อนข้างชัดเจน โดยคนที่ใช้บัตรจะมีแนวโน้มค่า Total_ct_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Avg_Utilization_Ratio พบว่าคนที่ใช้บัตรมีแนวโน้มที่ค่าเฉลี่ย Avg_Utilization_Ratio คนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Mount_on_book พบว่าที่ค่า Dependent_count 0-1 คนที่ใช้บัตรจะมีค่าเฉลี่ย Mount_on_book ต่ำกว่าคนที่ยกเลิกบัตร ในขณะที่ Dependent_count 2-5 ไม่สามารถแยกคนใช้บัตรและยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Total_Revolving_Bal พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Revolving_Bal ที่สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Total_Trans_Amt พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Amt ที่สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Total_Trans_Ct พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Ct ที่สูงกว่าคนที่ยกเลิกบัตร

#### เปรียบเทียบ Total_Relationship_Count

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Customer_Age", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Customer_Age", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Credit_Limit", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Credit_Limit", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Months_on_book", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Months_on_book", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Total_Relationship_Count",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Total_Relationship_Count", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Total_Relationship_Count", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Total_Relationship_Count กับ numerical ได้ผลดังนี้
- เมื่อเทียบกับอายุพบว่าเมื่อนำ Total_Relationship_Count ค่าต่างๆจะมีการกระจายตัวของอายุเป็น normal distribution ทั้งของคนที่ใช้บัตรและยกเลิกบัตรเครดิต
- เมื่อเปรียบเทียบกับcredits_limit พบว่าการกระจายตัวของแต่ละ Total_Relationship_Count จะมีลักษณะเหมือนและในขณที่ค่า Total_Relationship_Count 1-5 แสดงให้เห็นว่าผู่ที่ใช้บัตรเครดิตมีแนวโน้มค่าเฉลี่ยของของ credit_limits ที่มากกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับ Avg_Open_To_Buy พบว่ามีกระจายตัวใกล้เคียงกันและค่าเฉลี่ยของ Avg_Open_To_Buy มีค่าใกล้เคียงกันทั้งคนที่ใช้บัตรและยกเลิกบัตรทำให้ไม่สามารถแยกได้อย่างชัดเจน
- เมื่อเปรียบเทียบ Total_Amt_change_Q4_Q1 พบว่ามีการกระจายตัวใกล้เคียงกันในทุกๆค่าของ Total_Relationship_Count และกระจายตัวแบบ normal distribution
- เมื่อเปรียบเทียบ Total_ct_change_Q4_Q1 พบว่ามีการกระจายตัวของคนที่ใช้บัตรและยกเลิกบัตรเป็น normal distribution และเมื่อพิจารณาจากฐานนิยมและค่าเฉลี่ยจะสามารถแยกคนที่ใช้บัตรและยกเลิกบัตรได้ค่อนข้างชัดเจน โดยคนที่ใช้บัตรจะมีแนวโน้มค่า Total_ct_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Avg_Utilization_Ratio พบว่าคนที่ใช้บัตรมีแนวโน้มที่ค่าเฉลี่ย Avg_Utilization_Ratio คนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Mount_on_book พบว่าที่ค่า Total_Relationship_Count ไม่สามารถแยกคนใช้บัตรและยกเลิกบัตรได้อย่างชัดเจนเนื่อจากมีการกระจายตัวและค่าเฉลี่ยเหมือนกันในทุกๆค่าของ Total_Relationship_Count
- เมื่อเปรียบเทียบกับค่า Total_Revolving_Bal พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Revolving_Bal ที่สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Total_Trans_Amt พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Amt ที่สูงกว่าคนที่ยกเลิกบัตร โดยเฉพาะค่าที่ 1-2 ที่สามารถแยกได้อย่างชัดเจนซึ่เมื่อนำค่า Discrete ไปทำ Onehot encoding ให้ weigth ไม่เท่ากันในแต่ละค่าของ feature อาจได้ผลการทำนายที่ดีขึ้น
- เมื่อเปรียบเทียบกับค่า Total_Trans_Ct พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Ct ที่สูงกว่าคนที่ยกเลิกบัตร โดยเฉพาะค่าที่ 1-2 ที่สามารถแยกได้อย่างชัดเจนซึ่เมื่อนำค่า Discrete ไปทำ Onehot encoding ให้ weigth ไม่เท่ากันในแต่ละค่าของ feature อาจได้ผลการทำนายที่ดีขึ้น

#### เปรียบเทียบ Months_Inactive_12_mon

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Customer_Age", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Customer_Age", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Credit_Limit", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Credit_Limit", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Months_on_book", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Months_on_book", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Months_Inactive_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Months_Inactive_12_mon", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Months_Inactive_12_mon", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Months_inactive_12_mon กับ numerical ได้ผลดังนี้
- เมื่อเทียบกับอายุพบว่าเมื่อนำ Months_inactive_12_mon ค่าต่างๆจะมีการกระจายตัวของอายุเป็น normal distribution ทั้งของคนที่ใช้บัตรและยกเลิกบัตรเครดิต และจาก boxplot จะเห็นว่าค่า Months_inactive_12_mon 1-6 คนที่ยกเลิกบัตรจะมีแนวโน้มอายุที่เพิ่มขึ้นและค่า Months_inactive_12_mon 1,3,4,5,6 พบว่าคนที่ยกเลิกบัตรมักจะมีค่าเฉลี่ยอายุที่สูงกว่าคนที่ใช้บัตร
- เมื่อเปรียบเทียบกับcredits_limit พบว่าการกระจายตัวของแต่ละ Months_inactive_12_mon จะมีลักษณะเหมือนกันและค่าเฉลี่ยใกล้เคียงกันทั้งของคนที่ใช้บัตรและยกเลิกบัตร
- เมื่อเปรียบเทียบกับ Avg_Open_To_Buy พบว่ามีกระจายตัวใกล้เคียงกันและค่าเฉลี่ยของ Avg_Open_To_Buy บางค่าทำให้เราสามารถแยกประเภทคนได้แม้ไม่ชัดเจน
- เมื่อเปรียบเทียบ Total_Amt_change_Q4_Q1 พบว่ามีการกระจายตัวใกล้เคียงกันในทุกๆค่าของ Months_inactive_12_mon และกระจายตัวแบบ normal distribution โดยค่า Months_inactive_12_mon 0 มีจำนวนที่น้อยที่สุดแต่จะพบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย Months_inactive_12_mon ที่สูงกว่าคนยกเลิกบัตร โดยค่าอื่นๆใน feature ก็ให้ข้อสังเกตแบบเดียวกัน
- เมื่อเปรียบเทียบ Total_ct_change_Q4_Q1 พบว่ามีการกระจายตัวของคนที่ใช้บัตรและยกเลิกบัตรเป็น normal distribution และเมื่อพิจารณาจากฐานนิยมและค่าเฉลี่ยจะสามารถแยกคนที่ใช้บัตรและยกเลิกบัตรได้ค่อนข้างชัดเจน โดยคนที่ใช้บัตรจะมีแนวโน้มค่า Total_ct_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Avg_Utilization_Ratio พบว่าคนที่ใช้บัตรมีแนวโน้มที่ค่าเฉลี่ย Avg_Utilization_Ratio สูงคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Mount_on_book พบว่าที่ค่า Months_inactive_12_mon บางค่าสามารถให้ความแตกระหว่างคนที่ใช้บัตรได้แต่ก็จะเห็นว่าค่ามากสุดของคนที่ใช้บัตรและยกเลิกบัตรก็มีค่าใกล้เคียงกัน
- เมื่อเปรียบเทียบกับค่า Total_Revolving_Bal พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Revolving_Bal ที่สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Total_Trans_Amt พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Amt ที่สูงกว่าคนที่ยกเลิกบัตร
- เมื่อเปรียบเทียบกับค่า Total_Trans_Ct พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Ct ที่สูงกว่าคนที่ยกเลิกบัตร

#### เปรียบเทียบ Contacts_Count_12_mon

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Customer_Age", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Customer_Age", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Credit_Limit", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Credit_Limit", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Avg_Open_To_Buy", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Total_Amt_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Total_Ct_Chng_Q4_Q1", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Avg_Utilization_Ratio", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Months_on_book", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Months_on_book", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Total_Revolving_Bal", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Total_Trans_Amt", hue="Attrition_Flag", ax=axs[2])

In [ ]:
fig, axs = plt.subplots(1, 3, figsize=(25,10))
sns.countplot(data=df, x="Contacts_Count_12_mon",hue="Attrition_Flag", ax=axs[0])
sns.boxplot(data=df, x="Contacts_Count_12_mon", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[1])
sns.violinplot(data=df, x="Contacts_Count_12_mon", y="Total_Trans_Ct", hue="Attrition_Flag", ax=axs[2])

จากกราฟเปรียบเทียบ Contacts_Count_12_mon กับ numerical ได้ผลดังนี้
- เมื่อเทียบกับอายุพบว่าเมื่อนำ Contacts_Count_12_mon ค่าต่างๆจะมีการกระจายตัวของอายุเป็น normal distribution ทั้งของคนที่ใช้บัตรและยกเลิกบัตรเครดิต และจากกราฟจะเห็นว่าค่า Contacts_Count_12_mon 0 มื่อเทียบกับอายุจะสามารถบอกได้อย่างชัดเจนว่าคนที่อายูมากและมีค่า Contacts_Count_12_mon เป็น 0 มักจะเป็นคนที่ใช้บัตรรและพบว่าคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับ credits_limit พบว่าคนที่ใช้บัตรจะมีค่าเฉลี่ย credit_limits ที่สูงกว่าคนยกเลิกบัตรและพบว่าคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับ Avg_Open_To_Buy พบว่ามีกระจายตัวใกล้เคียงกันและค่าเฉลี่ยของ Avg_Open_To_Buy บางค่าทำให้เราสามารถแยกประเภทคนได้แม้ไม่ชัดเจนแต่พบว่าคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบ Total_Amt_change_Q4_Q1 บพว่าค่า Contacts_Count_12_mon 2-5 ค่าเฉลี่ย Contacts_Count_12_mon ของคนใช้บัตรจะมีมากกว่าคนที่ไม่ใช้บัตร และพบว่าคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบ Total_ct_change_Q4_Q1 พบว่ามีการกระจายตัวของคนที่ใช้บัตรและยกเลิกบัตรเป็น normal distribution โดยคนที่ใช้บัตรจะมีแนวโน้มค่า Total_ct_change_Q4_Q1 สูงกว่าคนที่ยกเลิกบัตร และพบว่าคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับค่า Avg_Utilization_Ratio พบว่าคนที่ใช้บัตรมีแนวโน้มที่ค่าเฉลี่ย Avg_Utilization_Ratio สูงกว่าคนที่ยกเลิกบัตร และพบว่าคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับค่า Mount_on_book พบว่าที่ค่าเฉลี่ยและการกระจายตัวของคนใช้บัตรและยกเลิกบัตรไม่ต่างกัน ยกเว้นคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับค่า Total_Revolving_Bal พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Revolving_Bal ที่สูงกว่าคนที่ยกเลิกบัตรและคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับค่า Total_Trans_Amt พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Amt ที่สูงกว่าคนที่ยกเลิกบัตรและคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด
- เมื่อเปรียบเทียบกับค่า Total_Trans_Ct พบว่าที่คนที่ใช้บัตรมีแนวโน้มที่ค่า Total_Trans_Ct ที่สูงกว่าคนที่ยกเลิกบัตรและคนที่ค่า Contacts_Count_12_mon เป็น 6 จะเป็นคนที่ยกเลิกบัตรทั้งหมด

### Summary Data Exploration

จากการทำ Data Exploration เราได้เห็นถึงความสัมพันธ์ต่างๆของข้อมูล เราจึงตั้งสมมติฐานว่า feature ที่มีความสำคัญต่อการวิเคราะห์มีดังนี้
- Total_Ct_Chng_Q4_Q1
- Avg_Utilization_Ratio
- Total_Revolving_Bal
- Total_Trans_Amt
- Total_trans_Ct
- Dependent_Count
- Total_Relationship_Count
- Contacts_count_12_mon
- Income_Category
- Card_Category

หลังจากนั้นเราจะทำการพิสูจน์สมมติฐานของเราโดยใช้ feature selection ทางสถิติและของ sklearn

## Data preprocessing

ก่อนที่จะทำ feature selection ผู้จัดทำจึงทำการตัดข้อมูลที่เป็น outlier เพื่อไม่ให้ข้อมูลเหล่านั้นมาทำให้การทำ feature selection ผิดพลาด โดยวิธีที่เลือกคือวิธี Interquartile Range

### Remove outlier

คำนวณ upper bound และ lower bound ของข้อมูลที่เป็น continuouse value และตัดข้อมูลที่อยู่นอกเหนือระหว่าง upper bound และ lower bound

In [ ]:
temp_df = df.copy()
for feature in contineous_columns:
    q3, q1 = np.percentile(df[feature], [75, 25])
    IQR = q3 - q1
    upper_bound = q3 + 1.5 * IQR
    lower_bound = q1 - 1.5 * IQR
    temp_df = temp_df[~((temp_df[feature] < (q1 - 1.5 * IQR)) | (temp_df[feature] > (q3 + 1.5 * IQR)))]

หาจำนวน outlier

In [ ]:
print(f"number of outlier {df.shape[0] - temp_df.shape[0]} instances")

In [ ]:
temp_df

visualize ข้อมูลใหม่เพื่อดูการกระจายตัวของข้อมูล

In [ ]:
compare_df = temp_df[contineous_columns]
compare_df = pd.concat((compare_df,temp_df["Attrition_Flag"]),axis=1)
sns.pairplot(compare_df,hue="Attrition_Flag")

โดยข้อมูลที่ได้หลังจากตัด outlier พบว่าการกระจายตัวข้อมูลบางตัวเปลี่ยนไป เช่น Total_Ct_Chng_Q4_Q1 มีความเป็น normal distribution มากขึ้น จากนั้นเช็กจำนวนของคนที่ใช้บัตรและยกเลิกบัตร

#### ดู distribution ของข้อมูล continuous

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[0], kde = True)

ทำการแก้ไข distribution โดยใช้ binning ทุก 10 เปอร์เซนไทล์

In [ ]:
deciles = temp_df[contineous_columns[0]].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])
sns.set_style('whitegrid')
fig, ax = plt.subplots()
temp_df[contineous_columns[0]].hist(ax=ax, bins=100)
for pos in deciles:
    handle = plt.axvline(pos, color='r')
ax.legend([handle], ['deciles'], fontsize=14)
ax.set_yscale('log')
ax.set_xscale('log')
ax.tick_params(labelsize=14)
ax.set_xlabel('Credits limit', fontsize=14)
ax.set_ylabel('Occurrence', fontsize=14)


In [ ]:
temp_val = pd.qcut(temp_df[contineous_columns[0]], 10, labels=False)
sns.displot(data=temp_val)
unique, counts = np.unique(temp_val, return_counts=True)
dict(zip(unique, counts))

In [ ]:
temp_df[contineous_columns[0]] = temp_val

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[1], kde = True)

In [ ]:
deciles = temp_df[contineous_columns[1]].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])
sns.set_style('whitegrid')
fig, ax = plt.subplots()
temp_df[contineous_columns[1]].hist(ax=ax, bins=100)
for pos in deciles:
    handle = plt.axvline(pos, color='r')
ax.legend([handle], ['deciles'], fontsize=14)
ax.set_yscale('log')
ax.set_xscale('log')
ax.tick_params(labelsize=14)
ax.set_xlabel('Avg_Open_To_Buy', fontsize=14)
ax.set_ylabel('Occurrence', fontsize=14)


In [ ]:
temp_val = pd.qcut(temp_df[contineous_columns[0]], 10, labels=False)
sns.displot(data=temp_val)
unique, counts = np.unique(temp_val, return_counts=True)
dict(zip(unique, counts))

In [ ]:
temp_df[contineous_columns[1]] = temp_val

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[2], kde = True)

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[3], kde = True)

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[4], kde = True)

In [ ]:
deciles = temp_df[contineous_columns[4]].quantile([.1, .2, .3, .4, .5, .6, .7, .8, .9])
sns.set_style('whitegrid')
fig, ax = plt.subplots()
temp_df[contineous_columns[4]].hist(ax=ax, bins=100)
for pos in deciles:
    handle = plt.axvline(pos, color='r')
ax.legend([handle], ['deciles'], fontsize=14)
ax.set_yscale('log')
ax.set_xscale('log')
ax.tick_params(labelsize=14)
ax.set_xlabel('Avg_Otilization_Ratio', fontsize=14)
ax.set_ylabel('Occurrence', fontsize=14)

In [ ]:
temp_val = pd.qcut(temp_df[contineous_columns[4]], 10, labels=False,duplicates= 'drop')
sns.displot(data=temp_val)
unique, counts = np.unique(temp_val, return_counts=True)
dict(zip(unique, counts))

In [ ]:
temp_df[contineous_columns[4]] = temp_val

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[5], kde = True)

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[6], kde = True)

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[7], kde = True)

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[8], kde = True)

In [ ]:
sns.displot(data=temp_df, x=contineous_columns[9], kde = True)

In [ ]:
temp_df["Attrition_Flag"].value_counts()

In [ ]:
dataset = temp_df.copy()

### Feature extraction

โดยการทำ feature selection เราได้เลือกมา 3 วิธีได้แก่ Random forest, chi2 และ PCA เพื่อทำการทดลองหาวิธีทำ feature selection ที่เหาะสมที่สุดสำหรับการทำ feature selection ในข้อมูลชุดนี้

เปลี่ยข้อมูล Discrete ให้เป็น Categorical และทำการ normalize ค่า Continuous

In [ ]:
for feature in discrete_columns:
    dataset[feature] = dataset[feature].astype("object")

In [ ]:
dataset.info()

In [ ]:
X = pd.get_dummies(dataset.drop(columns=["Attrition_Flag"]))
y = np.where(dataset["Attrition_Flag"] == "Existing Customer", 1, 0)

#### ใช้ Random forest ในการทำ feature selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.model_selection import train_test_split

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
for feature in contineous_columns:
    X[feature] = sc.fit_transform(X[feature].values.reshape(-1,1))

In [ ]:
X_data,X_test,y_data,y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
sel = SelectFromModel(RandomForestClassifier(n_estimators = 100))
sel.fit(X_data, y_data)

In [ ]:
sel.get_support()

In [ ]:
selected_feat= X_data.columns[(sel.get_support())]
len(selected_feat)

In [ ]:
print(selected_feat)

โดยจากการใช้ Random forest ทำ feature selection ได้ผลลัพธ์ feature ที่ตรงกับเราเลือกมา 6 feature ได้แก่ Total_Revolving_Bal, Total_Trans_Amt, Total_Trans_Ct, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio และ Total_Relationship_Count จากนั้นเราสร้างข้อมูลด้วย feature ที่ถูกเลือกเหล่านี้ไปทำการทดลองต่อไป

In [ ]:
rf_X_data = X_data[selected_feat]
rf_X_test = X_test[selected_feat]
rf_y_data = y_data
rf_y_test = y_test

#### ใช้ chi2 ในการทำ feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [ ]:
X = pd.get_dummies(dataset.drop(columns=["Attrition_Flag"]))
y = np.where(dataset["Attrition_Flag"] == "Existing Customer", 1, 0)

In [ ]:
X_data,X_test,y_data,y_test = train_test_split(X, y,test_size=0.2)

In [ ]:
selector = SelectKBest(chi2, k=13)     #This line creates the selector
x_new = selector.fit(X_data,y_data)             #This line fits the selector to the dataset, and select the features.

In [ ]:
col = selector.get_support(indices=True)   #all indices are saved in col.
X_data.columns[col]

โดยจากการใช้ chi2 ทำ feature selection ได้ผลลัพธ์ feature ที่ตรงกับเราเลือกมา 8 feature ได้แก่ Total_Revolving_Bal, Total_Trans_Amt, Total_Trans_Ct, Total_Ct_Chng_Q4_Q1, Avg_Utilization_Ratio, Total_Relationship_Count, Months_Inactive_12_mon  Contacts_Count_12_mon จากนั้นเราสร้างข้อมูลด้วย feature ที่ถูกเลือกเหล่านี้ไปทำการทดลองต่อไป

In [ ]:
chi2_X_data = X_data[X_data.columns[col]]
chi2_X_test = X_test[X_data.columns[col]]
chi2_y_data = y_data
chi2_y_test = y_test

#### ใช้ PCA ในการทำ feature selection

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components = len(X_data.columns))           # Create PCA transformer
x_pca = pca.fit_transform(X_data)          # Fit and transform PCA transformer to the dataset

print(pca.explained_variance_ratio_)  # This show the variance of each component.

จากการดูเปอเซนต์ของจำนวณ compunent axis ที่สามารถแปลงข้อมูลลงไปพบว่าที่ 3 component axis สามารถคลอบคุรมข้อมูลได้มากกว่า 90 % จึงใช้เลือก n_component ใช้เป็น 3

In [ ]:
pca = PCA(n_components = 3)
pca_X_data = pca.fit_transform(X_data)
pca_X_test = pca.transform(X_test)
pca_y_data = y_data
pca_y_test = y_test

## Model Construction and Analysis

จากการ visualize ข้อมูล พบว่าประเภทของคนที่ใช้บัตรและไม่ใช้บัตรไม่สามารถแยกได้ด้วยการขีดเส้นแบ่ง แต่อาจจะต้องนำข้อมูลเหล่านั้นไปมองในมุมใหม่หรือสร้าง Hyperplane เพื่อใช้ในการแบ่ง โดย model ที่เลือกมามีดังนี้
- Logistic regression โดยนำมาทดสอบกับข้อมูลที่ทำ feature selection ว่าสามารถแบ่งประเภทได้หรือไม่
- KNN เนื่องจากข้อมูลที่นำมาอาจจะไม่มี pattern ที่แบ่งแยกได้เป็นกลุ่มก้อนเดียวของ class นั้น จากที่สังเกตจาก pairplot จึงใช้ KNN เพื่อดูว่าข้อมูลตัวนี้อยู่ใกล้กลุ่มก้อนไหน
- Random forest ข้อมูลที่นำมาจากการทำเทียบข้อมูล Discrete และ Continuous พบว่าบาง feature จะมีค่าใน feature ที่สามารถใช้แยกประเภทได้มากว่าอันอื่น จึงเลือกใช้ model ที่ค่อยๆแบ่งประเภทจาก feature ไปเรื่อยๆ
- SVM เนื่องจากการ visaulize พบว่าไม่สามารถแยกประเภทได้ จึงเลือก SVM มาทำ hyperplane ในการแยกประเภท
- Naiev Bayes เนื่องจากการ visaulize พบว่าไม่สามารถแยกประเภทได้ จึงใช้ความน่าจะเป็นมาวิเคราะห์ว่าจะใช้บัตรหรือไม่ใช้บัตร
- Deep learning เมื่อต้องการหา pattern ของข้อมูลทั้งสอง class

### Logistic regression

สร้าง parameter สำหรับทำ hyperparameter tuning

In [ ]:
lr_param_grids = [
    {
        "alpha": np.power(10, np.arange(-4, 4, dtype=float)),
        "class_weight": [{0: 1, 1: 1}, {0: 1, 1: 1.1}, {0: 1, 1: 1.2}, {0: 1, 1: 1.3}, {0: 1, 1: 1.4}, {0: 1, 1: 1.5}, 'balanced']
    },
]

ทดลอง SGDclassifier กับข้อมูลที่ใช้ random forest ในการทำ feature selection

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import SGDClassifier

lr = SGDClassifier()

gs = GridSearchCV(lr, lr_param_grids, cv=10)
gs.fit(rf_X_data, rf_y_data)
print(gs.best_params_)

In [ ]:
lr = SGDClassifier(alpha=0.001, class_weight={0: 1, 1: 1.1})
lr.fit(rf_X_data, rf_y_data)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, ConfusionMatrixDisplay, confusion_matrix
cv = cross_val_score(lr,rf_X_data, rf_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(lr,rf_X_test, rf_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(rf_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_data, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(rf_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_test, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง SGDclassifier กับข้อมูลที่ใช้ chi2 ในการทำ feature selection

In [ ]:
lr = SGDClassifier()

gs = GridSearchCV(lr, lr_param_grids, cv=10)
gs.fit(chi2_X_data, chi2_y_data)
print(gs.best_params_)

In [ ]:
lr = SGDClassifier(alpha=0.01, class_weight={0: 1, 1: 1.4})
lr.fit(chi2_X_data, chi2_y_data)

In [ ]:
cv = cross_val_score(lr,chi2_X_data, chi2_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(lr,chi2_X_test, chi2_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(chi2_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_data, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(chi2_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_test, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง SGDclassifier กับข้อมูลที่ใช้ PCA ในการทำ feature selection

In [ ]:
lr = SGDClassifier()

gs = GridSearchCV(lr, lr_param_grids, cv=10)
gs.fit(pca_X_data, pca_y_data)
print(gs.best_params_)

In [ ]:
lr = SGDClassifier(alpha=1, class_weight={0: 1, 1: 1.2})
lr.fit(pca_X_data, pca_y_data)

In [ ]:
cv = cross_val_score(lr,pca_X_data, pca_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(lr,pca_X_test, pca_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(pca_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_data, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(pca_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_test, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

โดยจากการทดลอง Logistic Regression โมเดล SGDclassifier กับข้อมูลที่ใช้ feature selection ต่างๆทั้งสามรูปแบบจะเห็นได้ว่าการทำ SGDclassifier กับข้อมูลที่ใช้ PCA ในการทำ feature selection ให้ค่า recall มากที่สุดอยู่ที่ 0.64

### KNN

สร้าง parameter สำหรับทำ hyperparameter tuning

In [ ]:
knn_param_grids = [
    {
        "n_neighbors": [3, 5, 7, 9, 11, 13],
    },
]

ทดลอง knn กับข้อมูลที่ใช้ random forest ในการทำ feature selection

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn = KNeighborsClassifier()

gs = GridSearchCV(knn, knn_param_grids, cv=10)
gs.fit(rf_X_data, rf_y_data)
print(gs.best_params_)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(rf_X_data, rf_y_data)

In [ ]:
cv = cross_val_score(knn,rf_X_data, rf_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(knn,rf_X_test, rf_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(rf_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_data, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(rf_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_test, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง knn กับข้อมูลที่ใช้ chi2 ในการทำ feature selection

In [ ]:
knn = KNeighborsClassifier()

gs = GridSearchCV(knn, knn_param_grids, cv=10)
gs.fit(chi2_X_data, chi2_y_data)
print(gs.best_params_)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(chi2_X_data, chi2_y_data)

In [ ]:
cv = cross_val_score(knn,chi2_X_data, chi2_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(knn,chi2_X_test, chi2_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = knn.predict(chi2_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_data, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = knn.predict(chi2_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_test, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง knn กับข้อมูลที่ใช้ PCA ในการทำ feature selection

In [ ]:
knn = KNeighborsClassifier()

gs = GridSearchCV(knn, knn_param_grids, cv=10)
gs.fit(pca_X_data, pca_y_data)
print(gs.best_params_)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=13)
knn.fit(pca_X_data, pca_y_data)

In [ ]:
cv = cross_val_score(knn,pca_X_data, pca_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(knn,pca_X_test, pca_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(pca_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_data, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(pca_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_test, y_pred, labels=gs.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=gs.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

โดยจากการทดลอง KNN กับข้อมูลที่ใช้ feature selection ต่างๆทั้งสามรูปแบบจะเห็นได้ว่าการทำ KNN กับข้อมูลที่ใช้ chi2 และ PCA ในการทำ feature selection ให้ค่า recall มากที่สุดอยู่ที่ 0.65

### Random forest

สร้าง parameter สำหรับทำ hyperparameter tuning

In [ ]:
rf_param_grids = [
    {
        "criterion": ["gini", "entropy", "log_loss"],
        "class_weight": ["balanced", "balanced_subsample"] 
    },
]

ทดลอง Random forest กับข้อมูลที่ใช้ random forest ในการทำ feature selection

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier()

gs = GridSearchCV(rf, rf_param_grids, cv=10)
gs.fit(rf_X_data, rf_y_data)
print(gs.best_params_)

In [ ]:
rf = RandomForestClassifier(max_leaf_nodes=2,class_weight="balanced", criterion="log_loss")
rf.fit(rf_X_data, rf_y_data)

In [ ]:
cv = cross_val_score(rf,rf_X_data, rf_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(rf,rf_X_test, rf_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(rf_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_data, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(rf_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_test, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง Random forest กับข้อมูลที่ใช้ chi2 ในการทำ feature selection

In [ ]:
rf = RandomForestClassifier()

gs = GridSearchCV(rf, rf_param_grids, cv=10)
gs.fit(chi2_X_data, chi2_y_data)
print(gs.best_params_)

In [ ]:
rf = RandomForestClassifier(class_weight="balanced_subsample", criterion="log_loss")
rf.fit(chi2_X_data, chi2_y_data)

In [ ]:
cv = cross_val_score(rf,chi2_X_data, chi2_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(rf,chi2_X_test, chi2_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(chi2_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_data, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(chi2_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_test, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง Random forest กับข้อมูลที่ใช้ PCA ในการทำ feature selection

In [ ]:
rf = RandomForestClassifier()

gs = GridSearchCV(rf, rf_param_grids, cv=10)
gs.fit(pca_X_data, pca_y_data)
print(gs.best_params_)

In [ ]:
rf = RandomForestClassifier(class_weight="balanced_subsample", criterion="log_loss")
rf.fit(pca_X_data, pca_y_data)

In [ ]:
cv = cross_val_score(rf,pca_X_data, pca_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(rf,pca_X_test, pca_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(pca_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_data, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(pca_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_test, y_pred, labels=rf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=rf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

โดยจากการทดลอง Random Forest กับข้อมูลที่ใช้ feature selection ต่างๆทั้งสามรูปแบบจะเห็นได้ว่าการทำ Random Forest กับข้อมูลที่ใช้ random forest ในการทำ feature selection ให้ค่า recall มากที่สุดอยู่ที่ 0.81

### SVM

In [ ]:
svc_param_grids = [
    {
        "kernel": ["rbf", "poly"]
    },
]

ทดลอง SVM กับข้อมูลที่ใช้ random forest ในการทำ feature selection

In [ ]:
from sklearn.svm import SVC

clf = SVC()

gs = GridSearchCV(clf, svc_param_grids, cv=10, n_jobs=-1)
gs.fit(rf_X_data, rf_y_data)
print(gs.best_params_)

In [ ]:
clf = SVC(kernel="rbf")
clf.fit(rf_X_data, rf_y_data)

In [ ]:
cv = cross_val_score(clf,rf_X_data, rf_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(clf,rf_X_test, rf_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(rf_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_data, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(rf_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง SVM กับข้อมูลที่ใช้ chi2 ในการทำ feature selection

In [ ]:
clf = SVC()

gs = GridSearchCV(clf, svc_param_grids, cv=10, n_jobs=-1)
gs.fit(chi2_X_data, chi2_y_data)
print(gs.best_params_)

In [ ]:
clf = SVC(kernel="rbf")
clf.fit(chi2_X_data, chi2_y_data)

In [ ]:
cv = cross_val_score(clf,chi2_X_data, chi2_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(clf,chi2_X_test, chi2_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(chi2_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_data, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(chi2_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง SVM กับข้อมูลที่ใช้ PCA ในการทำ feature selection

In [ ]:
clf = SVC()

gs = GridSearchCV(clf, svc_param_grids, cv=10, n_jobs=-1)
gs.fit(pca_X_data, pca_y_data)
print(gs.best_params_)

In [ ]:
clf = SVC(kernel="rbf")
clf.fit(pca_X_data, pca_y_data)

In [ ]:
cv = cross_val_score(clf,pca_X_data, pca_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(clf,pca_X_test, pca_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = gs.predict(pca_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_data, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = gs.predict(pca_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

โดยจากการทดลอง SVM กับข้อมูลที่ใช้ feature selection ต่างๆทั้งสามรูปแบบจะเห็นได้ว่าการทำ SVM กับข้อมูลที่ใช้ random forest ในการทำ feature selection ให้ค่า recall มากที่สุดอยู่ที่ 0.68

### Naive bayes

ทดลอง Naive bayes กับข้อมูลที่ใช้ random forest ในการทำ feature selection

In [ ]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()

clf.fit(rf_X_data, rf_y_data)

In [ ]:
cv = cross_val_score(clf,rf_X_data, rf_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(clf,rf_X_test, rf_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = clf.predict(rf_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_data, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = clf.predict(rf_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง Naive bayes กับข้อมูลที่ใช้ chi2 ในการทำ feature selection

In [ ]:
clf = GaussianNB()

clf.fit(chi2_X_data, chi2_y_data)

In [ ]:
cv = cross_val_score(clf,chi2_X_data, chi2_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(clf,chi2_X_test, chi2_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = clf.predict(chi2_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_data, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = clf.predict(chi2_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลอง Naive bayes กับข้อมูลที่ใช้ PCA ในการทำ feature selection

In [ ]:
clf = GaussianNB()

clf.fit(pca_X_data, pca_y_data)

In [ ]:
cv = cross_val_score(clf,pca_X_data, pca_y_data,cv=10,n_jobs=-1)
print(f"train accuracy: {np.mean(cv)} +/- {np.std(cv)}")
cv = cross_val_score(clf,pca_X_test, pca_y_test,cv=10,n_jobs=-1)
print(f"test accuracy: {np.mean(cv)} +/- {np.std(cv)}")
y_pred = clf.predict(pca_X_data)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_data, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = clf.predict(pca_X_test)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_test, y_pred, labels=clf.classes_)
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=clf.classes_)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

โดยจากการทดลอง Naive bayes กับข้อมูลที่ใช้ feature selection ต่างๆทั้งสามรูปแบบจะเห็นได้ว่าการทำ Naive bayes กับข้อมูลที่ใช้ random forest ในการทำ feature selection จะให้ค่า recall มากที่สุดอยู่ที่ 0.64

### Deep learning

ทดลองสร้าง Model deep learning 

In [ ]:
from tensorflow import keras

inputs = keras.Input(shape=(rf_X_data.shape[1],))
x = keras.layers.Dense(32, activation='tanh')(inputs)
x = keras.layers.Dense(16, activation='tanh')(x)
x = keras.layers.Dense(8, activation='tanh')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=["accuracy"],
)

ทดลองกับข้อมูลที่ทำ feature selection ด้วย Random forest

In [ ]:
X_train = np.asarray(rf_X_data).astype('float32')
history = model.fit(X_train, rf_y_data, batch_size=16, epochs=50, validation_split=0.2)

test_scores = model.evaluate(rf_X_test, rf_y_test)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(rf_X_data)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_data, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = model.predict(rf_X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("Classification report \n=======================")
print(classification_report(y_true=rf_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(rf_y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลองกับข้อมูลที่ทำ feature selection ด้วย chi2

In [ ]:
inputs = keras.Input(shape=(chi2_X_data.shape[1],))
x = keras.layers.Dense(32, activation='tanh')(inputs)
x = keras.layers.Dense(16, activation='tanh')(x)
x = keras.layers.Dense(8, activation='tanh')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=["accuracy"],
)

In [ ]:
X_train = np.asarray(chi2_X_data).astype('float32')
history = model.fit(X_train, chi2_y_data, batch_size=16, epochs=50, validation_split=0.2)

test_scores = model.evaluate(chi2_X_test, chi2_y_test)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(chi2_X_data)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_data, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = model.predict(chi2_X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("Classification report \n=======================")
print(classification_report(y_true=chi2_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(chi2_y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

ทดลองกับข้อมูลที่ทำ feature selection ด้วย PCA

In [ ]:
inputs = keras.Input(shape=(pca_X_data.shape[1],))
x = keras.layers.Dense(32, activation='tanh')(inputs)
x = keras.layers.Dense(16, activation='tanh')(x)
x = keras.layers.Dense(8, activation='tanh')(x)
x = keras.layers.Dropout(0.5)(x)
outputs = keras.layers.Dense(1, activation='sigmoid')(x)
model = keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
model.summary()

In [ ]:
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(
    loss='binary_crossentropy',
    optimizer=opt,
    metrics=["accuracy"],
)

In [ ]:
X_train = np.asarray(pca_X_data).astype('float32')
history = model.fit(X_train, pca_y_data, batch_size=16, epochs=50, validation_split=0.2)

test_scores = model.evaluate(pca_X_test, pca_y_test)
print("Test loss:", test_scores[0])
print("Test accuracy:", test_scores[1])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
y_pred = model.predict(pca_X_data)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_data, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_data, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion matrix of training set')
plt.show()
print("\n=======================")

In [ ]:
y_pred = model.predict(pca_X_test)
y_pred = np.where(y_pred >= 0.5, 1, 0)
print("Classification report \n=======================")
print(classification_report(y_true=pca_y_test, y_pred=y_pred))
print("Confusion matrix \n=======================")
cm = confusion_matrix(pca_y_test, y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot()
plt.title('Confusion matrix of testing set')
plt.show()
print("\n=======================")

โดยจากการทดลอง Deep Learning กับข้อมูลที่ใช้ feature selection ต่างๆทั้งสามรูปแบบจะเห็นได้ว่าการทำ Deep Learning กับข้อมูลที่ใช้ random forest ในการทำ feature selection จะให้ค่า recall มากที่สุดอยู่ที่ 0.79

### Conclusion
1. จากผลการทดลองพบว่าการทำ feature selection ด้วย PCA จะได้ความแม่นยำที่ต่ำกว่าการทำด้วย Random forest กับ chi2 ในแต่ละโมเดล โดยผลที่เกิดขึ้นเกิดมาจากการที่ PCA ทำการแปลงค่าข้อมูลเดิมให้ไปอยู่ในปริภูมิแบบใหม่ทำให้ลักษณะบางอย่างของข้อมูลมีการสูญหาย
2. จากการทดลองใช้ SVM พบว่าได้ผลลัพธ์ที่ค่อนข้างแม่นยำในกรณีที่ใช้ random forest ทำ feature selection เนื่องจากข้อมูลอาจสามารถแปลงไปในปริภูมิใหม่โดยใช้ kernel method ของ SVM แล้วใช้ hyperplane แบ่งได้
3. จากการทดลองโมเดล Random Forest กับข้อมูลที่ใช้ Feature Selection ทั้งสามรูปแบบให้ผล recall โดยรวมออกมาดีที่สุด
4. จากการทดลองโมเดล กับข้อมูลที่ใช้ Random Forest ในการทำ Feature Selection ให้ผล recall โดยรวมกับโมเดลทั้ง 6 ดีที่สุด ดังนั้นหากมีความต้องการจะ predict ลูกค้าที่จะเลิกบัตรเครดิตไม่ให้ผิดพลาดมาก โมเดลที่ train มายังเหมาะสมที่จะนำไปใช้